In [1]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio

import config as cfg

In [2]:
df_fl = pd.read_csv('../data/flights_samp.csv')
df_ap = pd.read_csv('../data/airports.csv')

In [3]:
host= cfg.db['host']
port = cfg.db['port']
user = cfg.db['user']
pwd = cfg.db['pwd']
database= cfg.db['database']


con = psycopg2.connect(database=database, user=user, password=pwd, host=host, port=port)

print("Database opened successfully")

Database opened successfully


In [29]:
sql = '''
SELECT * from flights 
where fl_date >= '2018-12-27' and fl_date <= '2019-01-12';
'''

# data = sqlio.read_sql_query(sql, con)

In [31]:
data.to_csv('flights.csv', index=False)

In [33]:
data_samp = data.sample(n = 5000, random_state=0).reset_index()

In [35]:
data_samp.to_csv('flights_samp.csv', index=False)

In [1]:
def to_hhmmss(df_time_col):
    '''
    Change the format of the time in hhmm
    to hh:mm:ss, where ss is 00 in this case
    Input: `df_time_col`: a Pandas Series
    Return: a Pandas Series
    '''
    
    hhmm = []
    crs_hm = df_time_col.astype('str')
    for t in crs_hm:
        if len(t) == 1:
            hhmm.append('0' + t + ':00:00')
        elif (len(t) == 2) & (t < '24'):
            hhmm.append(t + ':00:00')
        elif (len(t) == 2) & (t > '24'):
            hhmm.append('00:' + t + ':00')
        elif len(t) == 3:
            hhmm.append('0' + t[0] + ':' + t[1:] + ':00')
        else:
            hhmm.append(t[:2] + ':' + t[2:] + ":00")
    return hhmm

In [ ]:
df['crs_dep_time_hhmm'] = to_hhmmss(df.crs_dep_time)

In [ ]:
def to_city_state(city_state_col):
    '''
    Change the format of `city1/city2, state`
    to `city1, state` 
    Input: A Pandas Series
    Output: A Pandas Series
    '''
    return pd.Series(map(lambda x: x[0].split('/')[0] + ',' + x[-1].strip() ,
                    city_state_col.str.split(',')))

In [ ]:
df.origin_city_name = to_city_state(df.origin_city_name)

In [ ]:
def request_weather(df):
    '''
    request weather information
    input: a data frame in which
        - first column: time 
        - second column: location
        - third column: date
    output: a dictionary 
    '''
    unit_group="us"
    api_key = cfg.visual_crossing['api_key']
    requestUrl = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline'
    weather_dict = { 'conditions': [] }
    count = 1
    for row in df.values:
        print('Query Count: ', count)
        time = row[0]
        loc = row[1]
        date = row[2]
        url = requestUrl + '/' + loc + '/' + date + 'T' + time + '?key=' + api_key   
        res = requests.get(url)
        if res.status_code == 200:
            weather_json = res.json()
            try:
                condition = weather_json['days'][0]['conditions']
            except:
                condition = 'NA'
            else:        
                weather_dict['conditions'].append(condition)
        else:
            weather_dict['conditions'].append('NA')
            print('Error: ', res.status_code)    
        count += 1
    return weather_dict

In [6]:
key = cfg.rapid_api['key']
base_url = "https://dark-sky.p.rapidapi.com"
url = base_url + '/' + ll + ',' + date + 'T' + time

headers = {
    'x-rapidapi-host': "dark-sky.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{"latitude":37.774929,"longitude":-122.419418,"timezone":"America/Los_Angeles","currently":{"time":1550692800,"summary":"Mostly Cloudy","icon":"partly-cloudy-day","precipIntensity":0,"precipProbability":0,"temperature":53.12,"apparentTemperature":53.12,"dewPoint":38.76,"humidity":0.58,"pressure":1014,"windSpeed":8.54,"windGust":14.74,"windBearing":277,"cloudCover":0.64,"uvIndex":4,"visibility":9.997},"hourly":{"summary":"Partly cloudy throughout the day.","icon":"partly-cloudy-day","data":[{"time":1550649600,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":48.34,"apparentTemperature":45.72,"dewPoint":39.56,"humidity":0.72,"pressure":1019,"windSpeed":5.93,"windGust":8.93,"windBearing":275,"cloudCover":0.61,"uvIndex":0,"visibility":9.997},{"time":1550653200,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":48.1,"apparentTemperature":44.66,"dewPoint":38.79,"humidity":